In [1]:
import os
import numpy as np
import pandas as pd
import dask
import gev_bayes_utils as gevbu
import gev_utils as gevu

from utils import city_list, gev_metric_ids
from utils import roar_data_path as project_data_path
from utils import roar_code_path as project_code_path

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
%%time
df_res = gevu.fit_ensemble_gev_city(
    city='nyc',
    metric_id='max_pr',
    years=[2015,2100],
    stationary=False,
    fit_method="mle",
)

LOCA2 ACCESS-CM2 ssp245 r1i1p1f1
CPU times: user 1min 32s, sys: 6.57 s, total: 1min 38s
Wall time: 1min 39s


In [4]:
df = pd.read_csv('/storage/group/pches/default/users/dcl5300/conus_comparison_lafferty-etal-2024/metrics/cities/nyc_max_pr.csv')

In [12]:
df[(df['ensemble'] == "LOCA2") & (df['gcm'] == "ACCESS-CM2") & (df['ssp'] == "ssp585") & (df['member'] == "r1i1p1f1")]

,gcm,member,ssp,ensemble,time,pr
211,ACCESS-CM2,r1i1p1f1,ssp585,LOCA2,2015,72.366860
212,ACCESS-CM2,r1i1p1f1,ssp585,LOCA2,2016,130.542590
213,ACCESS-CM2,r1i1p1f1,ssp585,LOCA2,2017,33.221516
214,ACCESS-CM2,r1i1p1f1,ssp585,LOCA2,2018,105.367590
215,ACCESS-CM2,r1i1p1f1,ssp585,LOCA2,2019,119.503890
...,...,...,...,...,...,...
292,ACCESS-CM2,r1i1p1f1,ssp585,LOCA2,2096,89.893660
293,ACCESS-CM2,r1i1p1f1,ssp585,LOCA2,2097,141.842350
294,ACCESS-CM2,r1i1p1f1,ssp585,LOCA2,2098,161.756880
295,ACCESS-CM2,r1i1p1f1,ssp585,LOCA2,2099,62.660942


In [3]:
df_res

,ensemble,gcm,member,ssp,loc_intcp,loc_trend,scale,shape
0,GARD-LENS,CESM2-LENS,1001_01,ssp370,61.014769,0.034244,15.842700,0.018800
0,GARD-LENS,CESM2-LENS,1011_01,ssp370,51.886959,0.194009,14.271702,-0.021210
0,GARD-LENS,CESM2-LENS,1021_02,ssp370,52.478663,0.163337,15.903970,0.144401
0,GARD-LENS,CESM2-LENS,1031_02,ssp370,53.044515,0.089147,13.757716,0.124398
0,GARD-LENS,CESM2-LENS,1041_03,ssp370,54.530126,0.161226,12.495107,0.030617
...,...,...,...,...,...,...,...,...
0,STAR-ESDM,NorESM2-LM,r1i1p1f1,ssp585,55.059119,0.099097,13.664535,-0.062454
0,STAR-ESDM,NorESM2-MM,r1i1p1f1,ssp245,59.523240,-0.020819,14.718152,-0.020616
0,STAR-ESDM,NorESM2-MM,r1i1p1f1,ssp585,55.032456,0.165393,15.438808,-0.011397
0,STAR-ESDM,TaiESM1,r1i1p1f1,ssp245,63.598176,0.004237,16.668389,0.003958


In [53]:
return_period = 100
return_period_year = np.arange(2015, 2100, 10) - 2015
gevu.estimate_return_level(return_period, loc_intcp + (loc_trend * return_period_year), scale, shape)

array([423.25790419, 423.01852174, 422.77913929, 422.53975684,
       422.30037439, 422.06099194, 421.82160949, 421.58222703,
       421.34284458])

In [19]:
loc, scale, shape = gevu.fit_gev_city(
    city='nyc',
    metric_id='max_pr',
    ensemble='LOCA2',
    gcm='CanESM5',
    member='r1i1p1f1',
    ssp='ssp245',
    years=[2015,2100],
    stationary=True,
    fit_method="lmom",
    return_periods=None,
    return_period_years=None,
    )
print(loc, scale, shape)
return_period = 100
gevu.estimate_return_level(return_period, loc, scale, shape)

72.26545295623754 28.142357648948348 -0.216004525647387


293.89277710665345

In [2]:
# ############
# ### Dask ###
# ############
# from dask_jobqueue import SLURMCluster

# cluster = SLURMCluster(
#     # account="pches",
#     account="open",
#     cores=3,
#     processes=1,
#     job_cpu=3,
#     memory="3GiB",
#     walltime="12:00:00",
#     job_script_prologue=[
#         f"export PYTHONPATH={project_code_path}/.venv/lib/python3.12/site-packages:$PYTHONPATH",  # Put venv first
#         "export JAX_PLATFORM_NAME=cpu",
#         "export XLA_FLAGS='--xla_force_host_platform_device_count=1'",
#         # Force PyTensor to not use caching at all
#         "export PYTENSOR_FLAGS='cxx=,",
#         "mode=FAST_COMPILE,",  # Less aggressive optimization but more stable
#         "allow_gc=True,",
#         "cache_size=0'"        # Disable caching completely
#     ],
#     death_timeout=60,
#     local_directory="/tmp"
# )

# cluster.scale(5)

# from dask.distributed import Client
# client = Client(cluster)
# client

In [2]:
############
### Dask ###
############
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    # account="pches",
    account="open",
    cores=1,
    processes=1,
    job_cpu=1,
    memory="10GiB",
    walltime="00:30:00",
)

cluster.scale(30)

from dask.distributed import Client
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.8.54:41391,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Get city timeseries

In [4]:
# # Run for all
# for city in city_list.keys():
#     for metric_id in gev_metric_ids:
#         gevbu.get_city_timeseries_all(city, metric_id)

## Fit Bayesian GEV

### Fit across ensemble

In [ ]:
%%time
# Fit info: non-stationary
future_years = [2015,2100]
stationary = False
return_periods = [100]

# Parallelize with dask delayed
delayed = []

# Loop thorugh all combos
for city in city_list.keys():
    for metric_id in gev_metric_ids:
        tmp = dask.delayed(gevbu.fit_bayesian_gev_ensemble)(
            city=city,
            metric_id=metric_id,
            years=future_years,
            stationary=stationary,
            shape_sigma=0.2,
            prior_identifier='shape_sigma_02',
            return_periods=return_periods,
        )
        delayed.append(tmp)

_ = dask.compute(*delayed)

In [2]:
# %%time
# # Fit info: stationary
# hist_years = [1950,2014]
# future_years = [2050,2100]
# stationary = True
# return_periods = [100]

# # Parallelize with dask delayed
# delayed = []

# # Loop thorugh all combos
# for city in city_list.keys():
#     for metric_id in gev_metric_ids:
#         for years in [hist_years, future_years]:
#             tmp = dask.delayed(gevbu.fit_bayesian_gev_ensemble)(
#                     city=city,
#                     metric_id=metric_id,
#                     years=years,
#                     stationary=stationary,
#                     return_periods=return_periods,
#                     dask=False
#             )
#             delayed.append(tmp)

# _ = dask.compute(*delayed)

### Gather results

In [3]:
%%time
# Loop thorugh all combos and store
store_path = f"{project_data_path}/extreme_value/cities/original_grid/bayes_combined/"

return_periods = [100]

prior_identifier = "shape_sigma_02"

stationary = False
stationary_string = "stat" if stationary else "nonstat"

# for city in city_list.keys():
for city in ['nyc', 'chicago', 'denver']:
    for metric_id in ['max_tasmax', 'max_pr', 'min_tasmin']:
    # for metric_id in gev_metric_ids:
        for years in [None, [2015,2100]]:
            # Check if done
            change_identifier = "" if years is None else f"_change_{years[0]}-{years[1]}"
            file_path = f"{store_path}/{city}_{metric_id}_{stationary_string}_{prior_identifier}{change_identifier}.csv"
            if os.path.exists(file_path):
                continue
            # Read
            df = gevbu.gather_bayesian_gev_results_all(
                city = city,
                metric_id = metric_id,
                return_periods = return_periods,
                stationary = stationary,
                prior_identifier = prior_identifier,
                years = years,
            )
            # Store
            df.to_csv(file_path, index=False)

CPU times: user 2min 39s, sys: 11.7 s, total: 2min 51s
Wall time: 15min 58s
